### Copyright (C) 2020 Sobhan Moradiyan Daghigh - All Rights Reserved

## Data Mining UniProj - no.2
#### 12/11/2020


In [1]:
# !pip install pyfpgrowth

In [2]:
# !pip install networkx

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyfpgrowth
import networkx as nx

### Reading the databases

In [4]:
orders = pd.read_csv(r"./dataset/orders.csv")

In [5]:
orders.head()

,ID_Order,ID_Customer,ID_Item,DateTime_CartFinalize,Amount_Gross_Order,city_name_fa,Quantity_item
0,2714054,469662,21386,2015-10-15 08:50:56.000,597982.0,محمود آباد,1.0
1,11104039,3063877,248497,2018-02-11 00:29:26.000,980000.0,خرمدره,1.0
2,4228130,3184893,50144,2016-06-14 00:30:08.000,229358.0,قرچک,1.0
3,22225624,6888562,70208,2018-09-03 14:37:19.000,16514.0,قم,1.0
4,4068771,2533490,67627,2016-05-21 11:51:02.000,133028.0,تهران,1.0


In [6]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID_Order               200000 non-null  int64  
 1   ID_Customer            200000 non-null  int64  
 2   ID_Item                200000 non-null  int64  
 3   DateTime_CartFinalize  200000 non-null  object 
 4   Amount_Gross_Order     200000 non-null  float64
 5   city_name_fa           200000 non-null  object 
 6   Quantity_item          200000 non-null  float64
dtypes: float64(2), int64(3), object(2)
memory usage: 10.7+ MB


### Ok now try to put each item in its transaction and make a list of transactions

In [28]:
grps = orders.groupby(by=['ID_Customer'])
transactions = []
for grp, pdf in grps:
    transactions.append(pdf['ID_Item'].tolist())
    

print("count of transactions:", len(transactions))
transactions




# grps = orders.groupby(by=['ID_Customer'])
# transactions = []
# for grp, pdf in grps:
#     transactions.append([pdf['city_name_fa'].tolist()[0], pdf['ID_Item'].tolist()])
    

# print("count of transactions:", len(transactions))
# transactions[5:15]

count of transactions: 151634


[[42545,
  361304,
  1523292,
  1671385,
  369749,
  1711488,
  5265,
  75068,
  80046,
  1771143,
  81692],
 [43418],
 [137418, 242763, 295740],
 [46854],
 [76901, 1390734, 87941],
 [2015151],
 [189732, 4631],
 [767994],
 [61363, 91566, 20173],
 [180116],
 [294942],
 [107583],
 [41842],
 [254822],
 [106303],
 [12048],
 [189695],
 [667294],
 [133070, 28236, 114994],
 [64039, 176246, 1352580, 73772],
 [116349, 104400],
 [90646, 9604, 93735, 81785, 183173, 1379404],
 [103256],
 [32109, 300689, 9780],
 [167665],
 [34150, 231547],
 [194997, 90640, 548393],
 [801172, 608800],
 [21001, 212460, 272044, 84546, 513728],
 [94547],
 [207302],
 [150959],
 [178279, 75551, 175451],
 [126206],
 [1054914],
 [83141],
 [23266],
 [422279, 1053717],
 [47187],
 [37031],
 [682948],
 [74033],
 [66743],
 [58803],
 [172604],
 [225449],
 [88768],
 [452511],
 [43118],
 [1826583],
 [222187, 75047, 65304, 124161, 762279],
 [12129, 32759],
 [216728],
 [70044],
 [282536],
 [77443, 1085613, 22105],
 [350147],
 [6299]

### Frequent patterns with support_threshold of 2

In [47]:
pattern = pyfpgrowth.find_frequent_patterns(transactions, 10)
pattern

{(37031,): 10,
 (124161,): 10,
 (58753,): 10,
 (75045,): 10,
 (172932,): 10,
 (173601,): 10,
 (380021,): 10,
 (102994,): 10,
 (62614,): 10,
 (20763,): 10,
 (75500,): 10,
 (167936,): 10,
 (292730,): 10,
 (37629,): 10,
 (138288,): 10,
 (392058,): 10,
 (14061,): 10,
 (165624,): 10,
 (181874,): 10,
 (42601,): 10,
 (2983,): 10,
 (142639,): 10,
 (69646,): 10,
 (519952,): 10,
 (64599,): 10,
 (236701,): 10,
 (77651,): 10,
 (124019,): 10,
 (166273,): 10,
 (358572,): 10,
 (821842,): 10,
 (54449,): 10,
 (133591,): 10,
 (274090,): 10,
 (421902,): 10,
 (258311,): 10,
 (20155,): 10,
 (49974,): 10,
 (262048,): 10,
 (44977,): 10,
 (11571,): 10,
 (10594,): 10,
 (244673,): 10,
 (29481,): 10,
 (252435,): 10,
 (183177,): 10,
 (42505,): 10,
 (14407,): 10,
 (11641,): 10,
 (314959,): 10,
 (514821,): 10,
 (75134,): 10,
 (59168,): 10,
 (45565,): 10,
 (954051,): 10,
 (128612,): 10,
 (9003,): 10,
 (200075,): 10,
 (96331,): 10,
 (42562,): 10,
 (74981,): 10,
 (155600,): 10,
 (127846,): 10,
 (1357428,): 10,
 (70010

In [27]:
orders['ID_Order'].nunique()

197631